<a href="https://colab.research.google.com/github/harshsinha-12/HackGPT/blob/main/HackGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
!pip install OpenAI
!pip install streamlit
!pip install langchain
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
# Facebook AI Similarity Search
!pip install unstructured
!pip install tiktoken
!pip install faiss-cpu

In [ ]:
os.environ['OPENAI_API_KEY'] = "sk-" #Enter your OPENAI API KEY

In [ ]:
# Initialise LLM with required paramerters

llm = OpenAI(temperature = 0.9, max_tokens = 1000)

loaders = UnstructuredURLLoader(
    urls = ["https://www.livemint.com/market/stock-market-news/nifty-50-sensex-today-what-to-expect-from-stock-market-indices-in-trade-on-october-27-11698372233576.html"])
data = loaders.load() #News Article Link

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

# As data is of type documents we can directly use split_docs over split_text

docs = text_splitter.split_documents(data)

In [ ]:
embeddings = OpenAIEmbeddings()
vectorindex_openai = FAISS.from_documents(docs, embeddings)

In [ ]:
# Starting vector index create in local

file_path = "/content/drive/MyDrive/Data Sets for Data Visualisation/faiss_store_openai.pkl"
with open(file_path, "wb") as f:
  pickle.dump(vectorindex_openai, f)

In [ ]:
if os.path.exists(file_path):
  with open(file_path, "rb") as f:
    vectorIndex = pickle.load(f)

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm = llm, retriever = vectorIndex.as_retriever())

In [ ]:
query = "What to expect from Nifty 50 today?"

langchain.debug = True

chain({"question" : query}, return_only_outputs = True)

#each chunk in seperate llm

[chain/start] [1:chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "What to expect from Nifty 50 today?"
}
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "He believes the short-term trend of Nifty 50 continues to be negative. However, having moved into the oversold region, there is a chance of upside bounce occurring from the lows.\n\n(Exciting news! Mint is now on WhatsApp Channels  Subscribe today by clicking the link and stay updated with the latest financial insights! Click here!)\n\nHere’s what to expect from Nifty and Sensex today:\n\nNifty 50\n\nThe Nifty 50 index slipped below 19,000 for the first time in four months, indicating a rising bearish condition.\n\n“The bearish crossover in the mome

{'answer': ' Based on analysis from market experts, it is expected that the Nifty 50 index will start with a flat opening and further corrections are possible from the current level.\n',
 'sources': 'https://www.livemint.com/market/stock-market-news/nifty-50-sensex-today-what-to-expect-from-stock-market-indices-in-trade-on-october-27-11698372233576.html'}